In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import time 

import folium
from folium.plugins import HeatMap
from IPython.display import IFrame

import os
import time
from selenium import webdriver
from PIL import Image 
import glob
#40.66, -73.95

In [23]:
data = pd.read_csv('../data/pedestrian_accidents.csv', low_memory=False)
data = data[data['CRASH TIME'].notna()]
data = data[data['LATITUDE'].notna()]
data = data[data['LONGITUDE'].notna()]

In [25]:
data['HOUR'] = pd.Series([datetime.strptime(time, '%H:%M').hour for time in data['CRASH TIME']], dtype='int')
data = data[data['HOUR'].notna()]
data['HOUR'] = data['HOUR'].astype(int)

In [24]:
def embed_map(map, filename):
    map.save(filename)
    return IFrame(filename, width='100%', height='500px')

In [28]:
for i in range(data.HOUR.min(), data.HOUR.max()+1):
    df_geo = data.loc[(data.HOUR == i)][['LATITUDE', 'LONGITUDE']].copy()
    map_5 = folium.Map(location=[40.66, -73.95], tiles='openstreetmap', zoom_start=10)
    HeatMap(data=df_geo, radius=12).add_to(map_5)
    title_html = f'''<h3 align="center" style="font-size:20px">
                    <b>Accidents during the day</b></h3>
                 '''
    map_5.get_root().html.add_child(folium.Element(title_html))
    embed_map(map_5, f'./maps/{i}_heatmap.html')

In [29]:
for i in range(0, 23):
    tmpurl=f'file:///C:/Users/Sudheendra Katikar/255/cmpe255-project-team1/maps/{i}_heatmap.html'
    browser = webdriver.Chrome(executable_path='./chromedriver.exe')
    browser.get(tmpurl)
    browser.save_screenshot(f'./maps/{i}_heatmap.png')
    browser.quit()

<ipython-input-29-c658d06806ec>:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path='./chromedriver.exe')


In [30]:
def png_to_gif(path_to_images, save_file_path, duration=1000):
    frames = []
    images = glob.glob(f'{path_to_images}')
    for i in sorted(images): 
        im = Image.open(i)
        im = im.resize((550,389),Image.ANTIALIAS)
        frames.append(im.copy())
    frames[0].save(f'{save_file_path}', format='GIF', append_images=frames[1:], save_all=True,
                   duration=duration, loop=0)

In [31]:
png_to_gif(path_to_images='./maps/*.png', save_file_path='./maps/pedestrian_heatmap.gif', duration=500)